# **1.1.1 Extractie rasterdata binnen buffers**
Om de invloed van landgebruik op de waarnemingen te onderzoeken is er data verzameld uit de raster kaart [Landelijk Grondgebruikbestand Nederland uit 2018 van de Wageningen Universiteit](https://www.wur.nl/nl/Onderzoek-Resultaten/Onderzoeksinstituten/Environmental-Research/Faciliteiten-tools/Kaarten-en-GIS-bestanden/Landelijk-Grondgebruik-Nederland/Versies-bestanden/LGN2018.htm). Deze kaart geeft, met een nauwkeurigheid van 5 bij 5 meter, aan wat het landgebruik over heel Nederland is. Met behulp van dit script is rondom de locatie van iedere waarneming binnen een radius van 100 meter (verstelbaar) bekeken wat het landgebruik is volgens het eerder genoemd Landelijk Grondgebruikbestand Nederland. De output geeft het aandeel dat ieder landgebruik type inneemt binnen de buffer in procenten. Daarnaast bied het de mogelijkheid deze output te clusteren in vooraf gemaakte groepen.



In [6]:
# Laad de benodigde ondersteunende bibliotheken in
from shapely.geometry import shape, Point
from pyproj import CRS as ProjCRS
from rasterio.mask import mask
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import pyproj
import sys
import os

## Intro en settings
Het doel van deze code is om omgevingsdata rondom coördinaten op te halen. Deze coördinaten moeten gegeven worden in een Excel bestand (.xlsx). De locaties moeten gegeven worden in Rijksdriehoekscoördinaten (RD-coördinaten) waarbij de X en Y component in losse kolommen gegeven wordt. Het gegeven Excel bestand hoeft verder niet leeg van data te zijn en kan zelfs meerdere tabbladen bevatten. Om dit te accommoderen wordt wel gevraagd om de naam van het tabblad en de kolommen met de coördinaten op voorhand te selecteren.

**Excel:**

In [7]:
# Vul hieronder het pad naar je Excel bestand in. 
# Laat hierbij het "Path(r"..")" intact.
Excel_file = Path(r"Input_data/TestInput.xlsx")

# Geef hier de naam van de Excel sheet met de coördinaten voor de extractie.
Excel_sheet = "Sheet1"

# Geef hier de Kolommen van de Excel sheet met coördinaten. 
# Dit script werkt voor RD coördinaten en gaat er vanuit dat de X en Y coördinaten in losse kolommen staan
Excel_X_coördinaat = "RDX"
Excel_Y_coördinaat = "RDY"

# Defineer het coordinaten systeem van de observaties en van de rasterkaart
Waarnemingen_crs = pyproj.CRS("EPSG:28992")  # Amersfoort RD New
     


De eerder genoemde omgevingsdata is afkomstig van een, door de gebruiker aangeleverde, raster kaart (.tif). De resolutie van deze kaart hoeft geen specifieke nauwkeurigheid te hebben maar houd er rekening mee dat een hogere resolutie resulteert in een hogere nauwkeurigheid in de output.


Daarnaast is een Legenda nodig om de pixelwaarde van de rasterdata te kunnen interpreteren. Deze legenda wordt vaak in een los bestandstype (bijvoorbeeld .sld) opgeslagen. Dit type bestand is moeilijk in te lezen in python, er is daarom voor een handmatig alternatief gekozen met een legenda in de vorm van een Excel bestand wat door de gebruiker op voorhand aangemaakt moet worden. Open hiervoor de kaartlaag in een GIS programma naar keuze en kopieer de legenda zoals hier onder is voorgedaan. De kolom "Categorie" is bedoelt voor het clusteren van de output. Wanneer meerdere regels een gelijke waarde krijgen in deze kolom wordt een extra gezamenlijke kolom aangemaakt in de output waar deze waarden opgeteld in verwerkt worden. Deze kolom kan naar wens ook volledig weg gelaten worden wat verder geen gevolgen voor het functioneren van de code heeft.

<style>
td:first-child {{
    text-align: right;
}}
td:last-child {{
    text-align: left;
}}
</style>
<table>
<tr>
    <td>Label waarde als aangegeven bij de legenda</td>
    <td>Label naam als aangeven bij de legenda</td>
    <td>Optionele categorie waarbinnen de labels vallen</td>
</tr>
<tr>
    <td><b>Value</b></td>
    <td><b>Name</b></td>
    <td><b>Categorie <code style="color:red;">(Niet verplicht)</code></b></td>
</tr>
<tr>
    <td>47</td>
    <td>overig gras</td>
    <td>Gras</td>
</tr>
<tr>
    <td>61</td>
    <td>boomkwekerijen</td>
    <td>Agrarisch</td>
</tr>
<tr>
    <td>62</td>
    <td>fruitkwekerijen</td>
    <td>Agrarisch</td>
</tr>
<tr>
    <td>251</td>
    <td>hoofdinfrastructuur en spoorbaanlichamen</td>
    <td>Bebouwing</td>
</tr>
    <td>etc.</td>
    <td>etc.</td>
    <td>etc.</td>
</tr>



**Kaart / Shape file:**

In [8]:
# Vul hieronder het pad naar je raster kaart.tif in. 
# Laat hierbij het "Path(r"..")" intact.
buffer_kaart_pad = Path(r"Input_data/LGN2018/LGN2018.tif")

# Vul hieronder het pad naar de handgemaakte legenda van je kaart in. 
# Laat hierbij het "Path(r"..")" intact.
kaart_Legenda = Path(r"Input_data/List_LGN.xlsx")

Als laatste moet de buffer grootte opgegeven worden. Deze waarde wordt gelezen als de radius (halve diameter) van de cirkel om iedere waarneming heen in meters. 

<code style="color:red;"><b>LET OP:</b> Waarnemingen zonder (valide) Coördinaten of Waarnemingen die buiten de kaart vallen worden niet weergegeven in de output </code>

**Buffer:**

In [9]:
# Vul hier de Radius in voor de buffer. (In meters)
buffer_radius = 100

## Het script

In [10]:
# Laad de opgegeven bestanden in
Excel_df = pd.read_excel(io = Excel_file, sheet_name = Excel_sheet)
buffer_kaart = rasterio.open(buffer_kaart_pad)
kaart_Legenda_df = pd.read_excel(kaart_Legenda, header=0)

# Controle of coordinaten systemen hetzelfde zijn
if Waarnemingen_crs != buffer_kaart.crs:
    # Maakt een transformer
    transformer = pyproj.Transformer.from_crs(Waarnemingen_crs, buffer_kaart.crs, always_xy=True)
    # Voert de transformatie uit en maakt nieuwe kolommen
    Excel_df['Converted_X'], Excel_df['Converted_Y'] = transformer.transform(Excel_df[Excel_X_coördinaat], Excel_df[Excel_Y_coördinaat])
    # Leest de coördinaten van de waarnemingen in en converteer de coördinaten zodat ze later naar buffers omgezet kunnen worden
    Excel_buffers = gpd.GeoDataFrame(
        geometry = [
            Point(X, Y) for X, Y in zip(Excel_df['Converted_X'], Excel_df['Converted_Y'])
        ], crs=buffer_kaart.crs)
else:
    Excel_buffers = gpd.GeoDataFrame(
        geometry = [
            Point(X, Y) for X, Y in zip(Excel_df[Excel_X_coördinaat], Excel_df[Excel_Y_coördinaat])
        ], crs=buffer_kaart.crs)

# Genereert buffers rondom iedere waarneming met de aangegeven radius
Excel_buffers = Excel_buffers.buffer(buffer_radius)

# maakt een dictionary om output voor losse label aandeel per waarneming op te slaan
data_dict_name = {}
# maakt een dictionary om output voor gegroepeerde label aandeel per waarneming op te slaan
data_dict_Grouped = {}
# Variable om de tel van de print-functie bij te houden. 
# Begint bij -1 zodat er aan het begin van de loop direct +1 toegevoegd kan worden en 
# tegelijkertijd te kunnen compenseren voor het tellen vanaf 0.
counter2 = 0
Counter = -1
#Bepaalt de max waarde van de printfunctie
counter_max = len(Excel_buffers.index)

# Bekijkt datapunten per waarneming
for buffer in Excel_buffers:
    # houd bij welke buffer verwerkt wordt als een indicatie van voortgang in de vorm van een geprinte update
    Counter += 1
    print(f"working on buffer {Counter} of {counter_max - 1}")

    # Checkt of de buffer een waarde heeft. Wanneer een waarneming geen coördinaten heeft wordt er geen buffer gegenereerd in de eerdere stap maar dit veroorzaakt geen error.
    # om deze gevallen te kunnen afvangen word hier gecheckt of de buffer gegenereerd is.
    if buffer:
        # De "try:" word gebruikt om het overlap van de buffer en kaart te testen. Wanneer de kaart geen overlap vertoont met de buffer 
        # veroorzaakt dit een error. De "try:" zorgt dat de code hierdoor niet stopt maar deze waarde over kan slaan nadat het een error
        # tegenkomt.
        try:
            # genereert een GeoDataFrame(GDF) met alle "pixels" uit de kaartlaag die overlappen met de geselecteerde buffer
            kaart_buffer, _ = mask(buffer_kaart, [shape(buffer)], crop=True)

            # Checkt of er een verwijzing naar de huidige buffer is in data_dict_name en maakt deze anders aan in zowel data_dict_name als 
            # in data_dict_Grouped. 
            if Counter not in data_dict_name:
                data_dict_name[Counter] = {}
                data_dict_Grouped[Counter] = {}
                counter2 += 1
            
            # Kijkt per regel van de legenda hoe vaak de waarde uit deze regel voorkomt in het nieuw gemaakte GDF
            for index, row in kaart_Legenda_df.iterrows():
                aantal = np.count_nonzero(kaart_buffer == row["Value"])
                
                # Slaat het gevonden aantal op onder het labelnaam behorende tot deze waarde/regel
                data_dict_name[Counter][row["Name"]] = aantal
                
                # Controleert of er categorieën in de legenda aangegeven zijn. 
                # als dit het geval is word gecontroleerd of er een verwijzing is naar de categorie in de data_dict_Grouped variabelen onder de counter value.
                # en maakt deze anders aan met value=0 zodat meerdere verwijzingen naar een zelfde categorie elkaar niet overschrijven
                if "Categorie" in kaart_Legenda_df.columns and row["Categorie"]:
                    if not row["Categorie"] in data_dict_Grouped[Counter]:
                        data_dict_Grouped[Counter][row["Categorie"]] = 0
                    data_dict_Grouped[Counter][row["Categorie"]] += aantal
        except:
            # print een mogelijke verklaring voor het falen van de datacollectie en gaat dan veder naar de volgende waarde in de loop.
            print(f"buffer {Counter} mislukt, mogelijk geen overlap tussen buffer en kaartlaag")
            pass
    # Als er geen buffer gegenereerd is word een mogelijke verklaring van het probleem geprint en verder gegaan zonder data te genereren     
    else:
        print(f"buffer {Counter} mislukt, mogelijk geen coördinaten")

# geeft feedback wanneer er geen buffers worden aangemaakt
if counter2 == 0:
        print("!!!ERROR: Coördinatensysteem van de waarnemingen zijn verkeerd of de waarnemingen vallen niet binnen het raster!!!")
        sys.exit()

# print een proces update naar de gebruiker
print("Calculating...")

# genereert een dataframe van de data in dictionary vorm en maakt een kolom met totaal aantal pixels per waarneming
# zodat waardes omgerekend kunnen worden naar procenten.
output_name_df = pd.DataFrame(data_dict_name)
output_name_df.loc["Sum"] = output_name_df.sum()
output_name_df = output_name_df.T

# kopieert de index van output_name_df naar een leeg dataframe waar de omgerekende output in opgeslagen gaat worden. 
output_procent_df = pd.DataFrame(index=output_name_df.index)

# berekend voor iedere legenda waarde het aandeel van de totale buffer in procent en slaat deze op in de output dataframe
for index, row in kaart_Legenda_df.iterrows():
    output_procent_df[row["Name"]]= output_name_df[row["Name"]]/output_name_df["Sum"]

# Controleer of er categorieën in de legenda aangegeven zijn.
# Als dit het geval is maakt het de zelfde berekeningen als voor output_name_df maar voor de output_categorie_df
if  "Categorie" in kaart_Legenda_df.columns:
    output_categorie_df = pd.DataFrame(data_dict_Grouped)
    output_categorie_df.loc["Sum"] = output_categorie_df.sum()
    output_categorie_df = output_categorie_df.T

    for columns in output_categorie_df.columns:
        if not columns == "Sum":
            output_procent_df[f"C_{columns}"]= output_categorie_df[columns] / output_categorie_df["Sum"]

# print een proces update naar de gebruiker
print("Saving data...")

# Verwijderd de lege rijen uit de output_procent_df zodat deze in de volgende regel samengevoegd kan worden met originele data mislukte waarnemingen in de output weer te geven.
output_procent_df = output_procent_df.dropna(axis="index", how="all")
all_data = pd.merge(Excel_df, output_procent_df, left_index=True, right_index=True, how='inner')

# Checkt of er een Output mapje op de locatie van de code staat en maakt deze anders aan
if not os.path.exists("Output"):
    os.makedirs("Output")

# Slaat de output op in een Excel bestand in Output Dir. print de locatie van de output naar de gebruiker.
all_data.to_excel(rf"Output/{Excel_file.stem}_{buffer_kaart_pad.stem}_{buffer_radius}.xlsx")
print(f"file saved at: {os.getcwd()}/Output/{Excel_file.stem}_{buffer_kaart_pad.stem}_{buffer_radius}.xlsx")


working on buffer 0 of 2649
working on buffer 1 of 2649
working on buffer 2 of 2649
working on buffer 3 of 2649
working on buffer 4 of 2649
working on buffer 5 of 2649
working on buffer 6 of 2649
working on buffer 7 of 2649
working on buffer 8 of 2649
working on buffer 9 of 2649
working on buffer 10 of 2649
working on buffer 11 of 2649
working on buffer 12 of 2649
working on buffer 13 of 2649
working on buffer 14 of 2649
working on buffer 15 of 2649
working on buffer 16 of 2649
working on buffer 17 of 2649
working on buffer 18 of 2649
working on buffer 19 of 2649
working on buffer 20 of 2649
working on buffer 21 of 2649
working on buffer 22 of 2649
working on buffer 23 of 2649
working on buffer 24 of 2649
working on buffer 25 of 2649
working on buffer 26 of 2649
working on buffer 27 of 2649
working on buffer 28 of 2649
working on buffer 29 of 2649
working on buffer 30 of 2649
working on buffer 31 of 2649
working on buffer 32 of 2649
working on buffer 33 of 2649
working on buffer 34 of 